In [1]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time

In [2]:
def rastrigin(x):
    A = 10
    n = len(x)
    return -(A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n)))

In [3]:
def rosenbrock(x):
    a = 1
    b = 100
    return -((a - x[0])**2 + b*(x[1] - x[0]**2)**2)


In [4]:
def shaffer(x):
    return -( 0.5 + (np.sin((x[0]**2 +x[1]**2))**2-0.5)/(1.0 + 0.001*(x[0]**2+x[1]**2))**2)

In [5]:
class Individual:
    global_best_position=None
    global_best_fitness=None
    def __init__(self,bounds,velocity_calc,calc_fitness,age=0,life_expectancy=120):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        

        self.life_expectancy=life_expectancy
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        self.age=age

        self.velocity_calc=velocity_calc
        
        self.personal_best_fitness=self.fitness
        self.personal_best_position=self.position.copy()
        
        if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.update_fitness()
       
                
    def update_velocity(self):
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Individual.global_best_position - self.position)
        
        #if any([x!=0 for x in personal_direction]):
         #   personal_direction/=np.linalg.norm(personal_direction)
        #if any([x!=0 for x in global_direction]):
         #   global_direction/=np.linalg.norm(global_direction)
        

        
        cognitive_velocity =random.random() * self.velocity_calc["cognitive"](self.age,self.life_expectancy)*personal_direction#*np.linalg.norm(self.velocity) 
        social_velocity =random.random() * self.velocity_calc["group"](self.age,self.life_expectancy)* global_direction#*np.linalg.norm(self.velocity)
        inertia = self.velocity_calc["inertia"](self.age,self.life_expectancy) * self.velocity
        self.velocity = inertia + cognitive_velocity + social_velocity
        self.velocity=np.clip(self.velocity,2*self.lower_bounds,2*self.upper_bounds)
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
        if self.personal_best_fitness is None or self.fitness>self.personal_best_fitness:
            self.personal_best_fitness=self.fitness
            self.personal_best_position=self.position.copy()
            if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    def mutate(self,mut_prob):
        #TODO
        for i in range(len(self.position)):
            if random.random()<mut_prob:
                self.position[i]+=0.01*random.uniform(self.lower_bounds[i],self.upper_bounds[i])
            if random.random()<mut_prob:
                self.velocity[i]+=0.01*random.uniform(self.lower_bounds[i],self.upper_bounds[i])
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)

In [6]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position
    c1.velocity=(coef*p1.velocity + (1-coef)*p2.velocity)/2

    c2.position=coef*p2.position +(1-coef)*p1.position
    c2.velocity=(coef*p2.velocity +(1-coef)*p1.velocity)/2

    c1.age=0
    c2.age=0

In [7]:
def mutation(ind:Individual,mut_prob=0.01):
    #ind.mutate(mut_prob)
    pass

In [8]:
def selection(population,tournament_size):
    allowed=random.sample(population,tournament_size)
    return max(allowed,key=lambda x:x.fitness)

In [9]:
velocity_calculator={
    "cognitive": lambda age,life_expectancy:2*(1-age/life_expectancy),
    "group" : lambda age ,life_expectancy:2*(age/life_expectancy),
    "inertia" : lambda age , life_expectancy:(120/120)
}
vezbe_velocity_calc={
    "cognitive":lambda age,life_expectancy:0.7 ,
    "group":lambda age, life_expectancy:1.0 ,
    "inertia":lambda age ,life_expectancy:1.0
    
}
maxtime=10**(-1)
def psogahybrid(pop_size, num_dimensions, allowed_time,velocity_calc,fitness_calc,life_expectancy=120,tournament_size=4):
    bounds = [(-5.12, 5.12) for _ in range(num_dimensions)]
    
    population = [Individual(bounds=bounds,
                      calc_fitness=fitness_calc,life_expectancy=life_expectancy,velocity_calc=velocity_calc,age=i%life_expectancy
                      ) for i in range(pop_size)]
    #print(Individual.global_best_fitness)
    
    
    start=time.time()
    while True:
        #sorted(population,key=lambda x:-x.fitness)
        
        for i in range(len(population)):
               
            population[i].update_velocity()
            population[i].update_position()
            population[i].age+=1
            population[i].update_fitness()
            
            if population[i].age==life_expectancy:
                     
                toPairWith=selection(population,tournament_size)
                child1=Individual(bounds=bounds,calc_fitness=fitness_calc,velocity_calc=velocity_calc,life_expectancy=life_expectancy)
                child2=Individual(bounds=bounds,calc_fitness=fitness_calc,velocity_calc=velocity_calc,life_expectancy=life_expectancy)
                crossover(population[i],toPairWith,child1,child2)
             
                mutation(child1)
                mutation(child2)
                population[i]=max([child1,child2],key=lambda x:x.fitness)
                population[i].update_fitness()
        if time.time()-start>allowed_time:
                tmp=Individual.global_best_fitness
                Individual.global_best_fitness=None
                Individual.global_best_position=None
                return tmp
       

   

In [76]:
#nasa hibridna optimizacija
psogahybrid(pop_size=50, num_dimensions=2, allowed_time=0.1,velocity_calc=vezbe_velocity_calc,life_expectancy=10,fitness_calc=rosenbrock,tournament_size=10)

-6.010846528907097e-15

In [11]:
#goal driven generational learning

Ideja algoritma:
    Imamo populaciju jedinki koje imaju svoju poziciju u prostoru i svoje usmerenje
    Imaju age-broj iteracija koliko je jedinka ziva.
    Imaju life_expectancy-zivotni vek u broju iteracija
    Pusticemo svaku jedinku da se krece kroz prostor nalik na pso dok ne bude imala 
    life expectancy godina.
    Razlika u odnosu na pso je sto dinamicki racunamo koeficijente prema broju godina i
    zivotnog ocekivanja.
    Kada jedinka dostigne life expectancy godina koristimo tournament selection da bi odabrali jedinku
    sa kojom cemo da je ukrstimo kako bi je zamenila u populaciji.(od dva deteta biramo najbolje)
    to radimo dok ne prodjemo maximalni broj iteracija.
    Mozda bi mutacija dovela do prevelikog diverziteta.
    Mozda bi neki elitism doprineo.
    

In [12]:
times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000

In [13]:
print("Function : Shaffer")
print("Population size: " + "50")
print("Num dimensions: " + "2")
print("c_i: "+"0.7")
print("c_p: " + "1.0")
print("c_g: " + "1.0")
print("Life expectancy: " + "50")
for allowed_time in times:
    suma=0
    for _ in range(num_trys):
        suma+=psogahybrid(pop_size=50, num_dimensions=2, allowed_time=allowed_time,velocity_calc=vezbe_velocity_calc,life_expectancy=50,fitness_calc=shaffer,tournament_size=10)
    suma=-suma
    suma/=num_trys
    
    print("    Time : " + str(allowed_time))
    
    print("    Average mistake: " +str(suma))
    print("\n")

Function : Shaffer
Population size: 50
Num dimensions: 2
c_i: 0.7
c_p: 1.0
c_g: 1.0
Life expectancy: 50
    Time : 0.001
    Average mistake: 0.009014530432020886




KeyboardInterrupt: 